<a href="https://colab.research.google.com/github/tastany/adep/blob/main/cnnbarkw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install mne

In [ ]:
import mne

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
path="/content/gdrive/MyDrive/Yahya_Tastan_DR/dataset/brainproducts/adep/ulvi_bahce_dinamik_v2.vhdr"#YAHYA_MUTFAK_dinamik
raw = mne.io.read_raw_brainvision(path,preload=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Extracting parameters from /content/gdrive/MyDrive/Yahya_Tastan_DR/dataset/brainproducts/adep/ulvi_bahce_dinamik_v2.vhdr...
Setting channel info structure...


<ipython-input-3-46cb772aa547>:4: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  raw = mne.io.read_raw_brainvision(path,preload=True)


Reading 0 ... 672619  =      0.000 ...   672.619 secs...


In [ ]:
# Veriye önizleme yapın
#raw.plot(duration=10)

In [ ]:
raw.drop_channels(["x_dir", "y_dir", "z_dir"])
montage =  mne.channels.make_standard_montage('easycap-M1')
raw.set_montage(montage, match_case=False)

Measurement date,"July 17, 2023 11:54:13 GMT"
Experimenter,Unknown
Participant,Unknown
Digitized points,19 points
Good channels,16 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,1000.00 Hz
Highpass,0.16 Hz
Lowpass,70.00 Hz


In [ ]:
events, _ = mne.events_from_annotations(raw)

Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1', 'Stimulus/S  2']


In [ ]:
raw.filter(0.4,50)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.4 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.40
- Lower transition bandwidth: 0.40 Hz (-6 dB cutoff frequency: 0.20 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 8251 samples (8.251 s)



Measurement date,"July 17, 2023 11:54:13 GMT"
Experimenter,Unknown
Participant,Unknown
Digitized points,19 points
Good channels,16 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,1000.00 Hz
Highpass,0.40 Hz
Lowpass,50.00 Hz


In [ ]:
#raw.annotations

In [ ]:
#epochs.plot(events=events)

In [ ]:
#epochs.drop_bad(reject=dict(eeg=150e-6))

In [ ]:

tmin, tmax = -2, 1 #3
epochs = mne.Epochs(
    raw,
    events,
    dict(left=1, right=2),
    tmin,
    tmax,
    baseline=(None, 0),
    preload=True
)

Not setting metadata
48 matching events found
Setting baseline interval to [-2.0, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 48 events and 3001 original time points ...
0 bad epochs dropped


In [ ]:
#print(type(epochs.get_data()))


In [ ]:
import numpy as np

# Örnek verileri oluştur
X = epochs.get_data()
y = epochs.events[:, 2]
X.shape
kanal_sayisi=X.shape[1]
ornek_uzunlugu=X.shape[2]


<ipython-input-13-c0607d113c56>:4: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X = epochs.get_data()


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# Veri kümesi boyutları
num_samples = 48
num_channels = 16
sequence_length = 3001

# Örnek veri oluşturma
X = torch.randn(num_samples, num_channels, sequence_length)
y = torch.randint(0, 2, (num_samples,))

# Veriyi eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# TensorDataset ve DataLoader kullanarak veri kümesi oluşturma
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=num_channels, out_channels=16, kernel_size=3)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)  # MaxPooling katmanı eklendi
        self.conv2 = nn.Conv2d(16, 64, kernel_size=3)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)  # MaxPooling katmanı eklendi
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)  # MaxPooling katmanı eklendi
        self.fc1 = nn.Linear(128 * 374, 512)  # Düzleştirilmiş boyutu güncelle
        self.fc2 = nn.Linear(512, 2)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = self.pool1(x)  # MaxPooling katmanını ekleyin
        x = nn.functional.relu(self.conv2(x))
        x = self.pool2(x)  # MaxPooling katmanını ekleyin
        x = nn.functional.relu(self.conv3(x))
        x = self.pool3(x)  # MaxPooling katmanını ekleyin
        x = x.view(-1, 128 * 374)  # Düzleştirilmiş boyutu güncelle
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# Model, optimizer ve kayıp fonksiyonu tanımlama
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Modeli eğitme
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Her epoch sonunda eğitim kaybını yazdırma
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Modelin performansını değerlendirme
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy: {100 * correct / total:.2f}%')


RuntimeError: Calculated padded input size per channel: (2 x 748). Kernel size: (3 x 3). Kernel size can't be greater than actual input size

In [ ]:
import torch

# Veri kümesi boyutları
num_samples = 48
num_channels = 16
sequence_length = 3001

# Örnek veri oluşturma
X = torch.randn(num_samples, num_channels, sequence_length)

# Conv2d katmanlarından önceki boyutları kontrol etme
print("X.shape before conv layers:", X.shape)

# Conv2d katmanlarının çıkış boyutlarını hesaplama
conv1 = nn.Conv2d(in_channels=num_channels, out_channels=16, kernel_size=3)
conv2 = nn.Conv2d(16, 64, kernel_size=3)
conv3 = nn.Conv2d(64, 128, kernel_size=3)

X = conv1(X)
X = conv2(X)
X = conv3(X)

# Conv2d katmanlarından sonra boyutları kontrol etme
print("X.shape after conv layers:", X.shape)


X.shape before conv layers: torch.Size([48, 16, 3001])


RuntimeError: Given groups=1, weight of size [16, 16, 3, 3], expected input[1, 48, 16, 3001] to have 16 channels, but got 48 channels instead

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np

# Modelin tahminlerini alın
y_pred = model.predict(X_test_normalized)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Konfüzyon matrisini oluşturun
conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)

# Konfüzyon matrisini yazdırın
print("Konfüzyon Matrisi:")
print(conf_matrix)


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Hassasiyet, Duyarlılık ve F1 değerlerini hesaplayın
precision = precision_score(y_true_classes, y_pred_classes, average='weighted')
recall = recall_score(y_true_classes, y_pred_classes, average='weighted')
f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')

# Sonuçları yazdırın
print("Hassasiyet (Precision): {:.2f}".format(precision))
print("Duyarlılık (Recall): {:.2f}".format(recall))
print("F1 Değeri: {:.2f}".format(f1))


In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

# İkili sınıflandırma durumu için ROC eğrisi ve AUC hesapla
fpr, tpr, thresholds = roc_curve(y_true_classes, y_pred_classes)
roc_auc = roc_auc_score(y_true_classes, y_pred_classes)

# ROC eğrisini çizdir
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label='ROC Curve (AUC = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()


In [ ]:
from sklearn.metrics import classification_report

report = classification_report(y_true_classes, y_pred_classes)
print("Sınıflandırma Raporu:")
print(report)
